In [1]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from bootstrapping.bootstrapping import BootStrapping
from make_representations.representation_maker import Metagenomic16SRepresentation
from classifier.classical_classifiers import RFClassifier,SVM
from classifier.DNN import DNNMutliclass16S
import pandas as pd

Using TensorFlow backend.
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

# Crohn's Disease

### Ler fastaq

In [2]:
# LER OS FASTAQ

# fasta_files: arquivos fastaq
# mapping: orde na que foron lidos/iterados
#fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq')
fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq', 
                only_files=["ERR1368879.fastq", "ERR1368880.fastq", "ERR1368881.fastq", 
                            "ERR1368882.fastq", "ERR1368883.fastq", "ERR1368884.fastq",
                            "ERR1368885.fastq", "ERR1368886.fastq", "ERR1368887.fastq",
                            "ERR1368888.fastq", "ERR1368889.fastq", "ERR1368890.fastq"])

### 1. Bootstrapping

In [9]:
# CONSTRUIR OBXECTO PARA O BOOTSTRAPPING
'''
input_dir: input directory or a list of files
output_dir: a directory to generate the output files
sampling_sizes: a list of sampling sizes (N's)
n_resamples: number of resamples from each file (N_R = 10)
seqtype: file suffixes fastq or fasta etc.
M: number of files from the directory to make samplings from (M)
'''

BS=BootStrapping(fasta_files,'../../crohns_disease/results/bootstrapping/', seqtype='fastq',
                 sampling_sizes=[10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000], n_resamples=10, M=10)

In [ ]:
# add k-mer samplings
for k in [3,4,5,6,7,8]:
    print(k,'-mer bootstrapping..')
    BS.add_kmer_sampling(k)

In [19]:
# GARDAR RESULTADOS EN ARQUIVO 'bootstrapping_crohns_disease_prediction' NA RUTA QUE SE LLE INDICOU AO DEFINIR A INSTANCIA BS
BS.save_me('bootstrapping_crohns_disease_prediction')

### 2. k-mer representations

In [ ]:
# ENTRENAMENTO COS MELLORES TAMAÑOS DE K-MER (K) E OS SEUS CORRESPONDENTES TAMAÑOS DE MOSTRA (N) 
# (SEGÚN OS RESULTADOS DE BOOTSTRAPPING)

# dictionary of k-mers and their sampling values
# 6:[100,1000,2000,5000,10000,-1]  :  k-mer:[sampling values]      (-1 means using all sequences)
#sampling_dict={3:[20],4:[100],5:[500],6:[100,1000,2000,5000,10000,-1],7:[5000],8:[8000]}
sampling_dict={6:[5000]}

# number of cores
nump_p=20

# Para cada arquivo fastaq, creamos unha representación por cada posible combinación de tamaño de k-mer e tamaño de mostra, e gardamos os resultados
for k in sampling_dict.keys():  # for each k-mer
    for N in sampling_dict[k]:  # select a sampling size N
        print(k,'-mers with sampling size ',N)
        
        RS=Metagenomic16SRepresentation(fasta_files, mapping, N, nump_p)
        RS.generate_kmers_all(k, save='../../crohns_disease/datasets/PRUEBA-dataset_'+str(k)+'-mers_rate_complete1359_seq_'+str(N))

### 3. Classifers

#### Classical Classifiers

In [2]:
# CARGAR O ARQUIVO DE REPRESENTACIÓN K-MERS QUE MELLORES RESULTADOS DESE E OS RESULTADOS DE CLASIFICACIÓN REAIS

# X = matriz con datos:  (nº mostra, nº k-mers da secuencia) Abundancia relativa do k-mer na secuencia
#     en total temos 1359 mostras e 4096 6-mers
# Y = lista de resultados reais:  (CD , not CD)
#X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/dataset_6-mers_rate_complete1359_seq_5000.npz')
X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.npz')
#Y=FileUtility.load_list('../../crohns_disease/datasets/labels_disease_complete1359.txt')
Y=FileUtility.load_list('../../crohns_disease/datasets/PRUEBA-labels_disease_complete1359.txt')


# vemos os datos de X
#dataset=pd.read_csv("../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.csv")
#dataset

In [ ]:
# CREAR E ENTRENAR OS CLASIFICADORES E GARDAR OS RESULTADOS 
##### (results containing the best parameter, confusion matrix, best estimator)

################ Random Forest ################
MRF = RFClassifier(X, Y)
MRF.tune_and_eval('../../crohns_disease/results-PRUEBA/RFclassifier', n_fold=2)

In [ ]:
################ Support Vector Machine ################
MSVM = SVM(X, Y)
MSVM.tune_and_eval('../../crohns_disease/results-PRUEBA/SVMclassifier', n_fold=2)

#### DNN Classifier

In [2]:
# CARGAR O ARQUIVO DE REPRESENTACIÓN K-MERS QUE MELLORES RESULTADOS DESE E OS RESULTADOS DE CLASIFICACIÓN REAIS

#X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/dataset_6-mers_rate_complete1359_seq_5000.npz').toarray()
X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.npz').toarray()
#Y=FileUtility.load_list('../../crohns_disease/datasets/labels_disease_complete1359.txt')
Y=FileUtility.load_list('../../crohns_disease/datasets/PRUEBA-labels_disease_complete1359.txt')

In [ ]:
# CREAR E ENTRENAR OS CLASIFICADORES E GARDAR OS RESULTADOS 
##### (results containing the best parameter, confusion matrix, best estimator)

################ Multilayer Perceptron  ################
DNN=DNNMutliclass16S(X, Y, model_strct='mlp', model_arch=[1024,0.2,512,0.2,256,0.1,128,8])
#DNN.cross_validation('../../crohns_disease/results-PRUEBA/DNNclassifier', gpu_dev='1', n_fold=10, epochs=100, batch_size=1000)
DNN.tune_and_eval('../../crohns_disease/results-PRUEBA/DNNclassifier', gpu_dev='1', n_fold=2, epochs=1, batch_size=6)

In [5]:
#X_new=DNNMutliclass16S.make_activation_function(X, '../../crohns_disease/results-PRUEBA/DNNclassifier/weights_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.37.pickle')
X_new=DNN.make_activation_function()
X_new

array([[0.        , 0.01157627, 0.        , 0.00637048, 0.        ,
        0.        , 0.00362285, 0.01020442],
       [0.        , 0.01144924, 0.        , 0.00675407, 0.        ,
        0.        , 0.00399915, 0.01095819],
       [0.        , 0.01108021, 0.        , 0.00702202, 0.        ,
        0.        , 0.00364185, 0.01077139],
       [0.        , 0.01152687, 0.        , 0.00661017, 0.        ,
        0.        , 0.00361122, 0.0102833 ],
       [0.        , 0.0117835 , 0.        , 0.00686521, 0.        ,
        0.        , 0.0043022 , 0.01173091],
       [0.        , 0.01147059, 0.        , 0.00706169, 0.        ,
        0.        , 0.00391506, 0.01050185],
       [0.        , 0.01140304, 0.        , 0.00665884, 0.        ,
        0.        , 0.00353548, 0.01029341],
       [0.        , 0.01158554, 0.        , 0.00684084, 0.        ,
        0.        , 0.00413949, 0.0108822 ],
       [0.        , 0.01158609, 0.        , 0.0071018 , 0.        ,
        0.        , 0.003974

### Mostrar as métricas obtidas (as resumidas están nos .txt)

In [7]:
### RF
metrics=pd.read_csv("../../crohns_disease/results-PRUEBA/RFclassifier/all_metrics.csv")
metrics

,Unnamed: 0,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_f1_macro,mean_test_f1_micro,mean_test_precision_macro,mean_test_precision_micro,mean_test_recall_macro,mean_test_recall_micro,...,std_fit_time,std_score_time,std_test_accuracy,std_test_f1_macro,std_test_f1_micro,std_test_precision_macro,std_test_precision_micro,std_test_recall_macro,std_test_recall_micro,std_test_roc_auc
0,0,0.274773,0.048121,0.583333,0.500000,0.583333,0.500,0.583333,0.6250,0.583333,...,0.015310,0.008291,0.083333,0.166667,0.083333,0.250,0.083333,0.1250,0.083333,0.125000
1,1,0.392367,0.107916,0.583333,0.500000,0.583333,0.500,0.583333,0.6250,0.583333,...,0.007053,0.004524,0.083333,0.166667,0.083333,0.250,0.083333,0.1250,0.083333,0.222222
2,2,1.625224,0.407611,0.500000,0.409524,0.500000,0.475,0.500000,0.5625,0.500000,...,0.046510,0.030360,0.000000,0.076190,0.000000,0.225,0.000000,0.0625,0.000000,0.076389
3,3,4.355545,0.769961,0.583333,0.500000,0.583333,0.500,0.583333,0.6250,0.583333,...,0.123378,0.100070,0.083333,0.166667,0.083333,0.250,0.083333,0.1250,0.083333,0.013889


In [8]:
### SVM
metrics=pd.read_csv("../../crohns_disease/results-PRUEBA/SVMclassifier/all_metrics.csv")
metrics

,Unnamed: 0,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_f1_macro,mean_test_f1_micro,mean_test_precision_macro,mean_test_precision_micro,mean_test_recall_macro,mean_test_recall_micro,...,std_fit_time,std_score_time,std_test_accuracy,std_test_f1_macro,std_test_f1_micro,std_test_precision_macro,std_test_precision_micro,std_test_recall_macro,std_test_recall_micro,std_test_roc_auc
0,0,0.092845,0.015740,0.416667,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,...,0.015190,0.000325,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
1,1,0.106711,0.018778,0.583333,0.366667,0.583333,0.291667,0.583333,0.5000,0.583333,...,0.010007,0.003846,0.083333,0.033333,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
2,2,0.102203,0.014340,0.416667,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,...,0.011243,0.001954,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
3,3,0.091971,0.010812,0.416667,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,...,0.001941,0.000159,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
4,4,0.042563,0.012324,0.416667,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,...,0.003594,0.001625,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
5,5,0.038439,0.015243,0.416667,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,...,0.009554,0.007304,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
6,6,0.021215,0.020228,0.416667,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,...,0.002920,0.009681,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
7,7,0.030475,0.040213,0.500000,0.409524,0.500000,0.475000,0.500000,0.5625,0.500000,...,0.015342,0.001382,0.000000,0.076190,0.000000,0.225000,0.000000,0.0625,0.000000,0.104167
8,8,0.009235,0.013067,0.500000,0.409524,0.500000,0.475000,0.500000,0.5625,0.500000,...,0.000472,0.002111,0.000000,0.076190,0.000000,0.225000,0.000000,0.0625,0.000000,0.104167
9,9,0.005258,0.010967,0.500000,0.409524,0.500000,0.475000,0.500000,0.5625,0.500000,...,0.000844,0.000129,0.000000,0.076190,0.000000,0.225000,0.000000,0.0625,0.000000,0.104167


In [6]:
### MLP


# Probas varias

In [ ]:
# proba MLP
#z=FileUtility.load_obj('../../crohns_disease/results/DNNclassifier/weights_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')
#t=FileUtility.load_sparse_csr('../../crohns_disease/results/DNNclassifier/activationlayer_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

#print(z)

###########################################################################################

#[label_set, conf, best_score_, best_estimator_, cv_results_, best_params_, predictions]=FileUtility.load_obj('../../crohns_disease/results-PRUEBA/RFclassifier/all_results.pickle')

#results=pd.read_csv("../../crohns_disease/results-PRUEBA/RFclassifier/cv_results_.csv")

#results.columns

#results[['mean_test_f1_macro', 'mean_test_f1_micro', 'mean_test_precision_macro', 
#       'mean_test_precision_micro', 'mean_test_recall_macro', 'mean_test_recall_micro',
#       'std_test_f1_macro', 'std_test_f1_micro', 'std_test_precision_macro',
#       'std_test_precision_micro', 'std_test_recall_macro', 'std_test_recall_micro']]


#results.drop(['mean_fit_time', 'mean_score_time', 'std_fit_time', 'std_score_time', 
#              'param_max_features', 'param_min_samples_leaf', 'param_min_samples_split',
#              'param_criterion', 'param_n_estimators', 'rank_test_f1_macro', 'rank_test_f1_micro',
#              'rank_test_precision_macro', 'rank_test_precision_micro', 'rank_test_recall_macro',
#              'rank_test_recall_micro'], axis=1)

###########################################################################################

#[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, 
# (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-ARTICULO/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

###########################################################################################3

#print(sys.version)
#import pkg_resources
#installed_packages = pkg_resources.working_set
#installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
#for i in installed_packages])
#print(installed_packages_list)